Main Source https://towardsdatascience.com/find-highly-correlated-stocks-with-python-77eba4fd061b
HeatMap plot https://blog.patricktriest.com/analyzing-cryptocurrencies-python/

In [23]:
import numpy as np
import warnings
from pandas_datareader import data as pdr
import yfinance as yf
import datetime as dt
from yahoo_fin import stock_info as si
import pandas as pd
pd.set_option('display.max_rows', None)
warnings.filterwarnings("ignore")
yf.pdr_override()

num_of_years = 0.5
start = dt.date.today() - dt.timedelta(days = int(365.25*num_of_years))
end = dt.date.today()

#IMPORTANT you can change the tickers or merge the df with other stocks
#tickers = si.tickers_dow()
tickers = ['AAPL', 'TCEHY', 'NVDA', 'GOOG','QCOM', 'MSFT', 'FB']
tickers.append('BTC-USD')
tickers.append('ETH-USD')
tickers.append('SAND-USD')
tickers.append('MANA-USD')
tickers.append('MATIC-USD')
print(tickers)

['AAPL', 'TCEHY', 'NVDA', 'GOOG', 'QCOM', 'MSFT', 'FB', 'BTC-USD', 'ETH-USD', 'SAND-USD', 'MANA-USD', 'MATIC-USD']


In [24]:
dataset = pdr.get_data_yahoo(tickers, start, end)['Adj Close']
stocks_returns = np.log(dataset/dataset.shift(1))

print('\nCorrelation Matrix')
corr_matrix = stocks_returns.corr()
print (corr_matrix)

[*********************100%***********************]  12 of 12 completed

Correlation Matrix
               AAPL   BTC-USD   ETH-USD        FB      GOOG  MANA-USD  \
AAPL       1.000000  0.382504  0.443793  0.363859  0.602921  0.285432   
BTC-USD    0.382504  1.000000  0.864587  0.200248  0.357099  0.371469   
ETH-USD    0.443793  0.864587  1.000000  0.213480  0.352502  0.402345   
FB         0.363859  0.200248  0.213480  1.000000  0.522910  0.152265   
GOOG       0.602921  0.357099  0.352502  0.522910  1.000000  0.253291   
MANA-USD   0.285432  0.371469  0.402345  0.152265  0.253291  1.000000   
MATIC-USD  0.413843  0.688528  0.746574  0.137716  0.287587  0.262075   
MSFT       0.691236  0.448075  0.402399  0.516632  0.835081  0.353049   
NVDA       0.645738  0.383828  0.440120  0.444392  0.673325  0.397092   
QCOM       0.593491  0.189667  0.236508  0.446510  0.609278  0.141836   
SAND-USD   0.332886  0.451750  0.497483  0.154429  0.255171  0.778957   
TCEHY      0.288478  0.319540  0.

In [11]:
import plotly.graph_objs as go
import plotly.offline as py

def correlation_heatmap(df, title, absolute_bounds=True):
    '''Plot a correlation heatmap for the entire dataframe'''
    heatmap = go.Heatmap(
        z=df.corr(method='pearson').to_numpy(),
        x=df.columns,
        y=df.columns,
        colorbar=dict(title='Pearson Coefficient'),
    )
    
    layout = go.Layout(title=title)
    
    if absolute_bounds:
        heatmap['zmax'] = 1.0
        heatmap['zmin'] = -1.0
        
    fig = go.Figure(data=[heatmap], layout=layout)
    py.iplot(fig)

In [25]:
correlation_heatmap(stocks_returns, "Stocks Correlations")

In [7]:
def df_scatter(df, title, seperate_y_axis=False, y_axis_label='', scale='linear', initial_hide=False):
    '''Generate a scatter plot of the entire dataframe'''
    label_arr = list(df)
    series_arr = list(map(lambda col: df[col], label_arr))
    
    layout = go.Layout(
        title=title,
        legend=dict(orientation="h"),
        xaxis=dict(type='date'),
        yaxis=dict(
            title=y_axis_label,
            showticklabels= not seperate_y_axis,
            type=scale
        )
    )
    
    y_axis_config = dict(
        overlaying='y',
        showticklabels=False,
        type=scale )
    
    visibility = True
    if initial_hide:
        visibility = 'legendonly'
        
    # Form Trace For Each Series
    trace_arr = []
    for index, series in enumerate(series_arr):
        trace = go.Scatter(
            x=series.index, 
            y=series, 
            name=label_arr[index],
            visible=visibility
        )
        
        # Add seperate axis for the series
        if seperate_y_axis:
            trace['yaxis'] = 'y{}'.format(index + 1)
            layout['yaxis{}'.format(index + 1)] = y_axis_config    
        trace_arr.append(trace)

    fig = go.Figure(data=trace_arr, layout=layout)
    py.iplot(fig)

In [26]:
# Plot all of the BTC exchange prices
df_scatter(stocks_returns, 'Stock Prices Bonds and BTC')